In [1]:
import numpy as np
import time
import torch
import torch.nn as nn

In [17]:
#Optional rknn-toolkit2 installs onnx anyway
#!pip3 install onnx
#check python version and download rknn toolkit from rockchip repo https://github.com/airockchip/rknn-toolkit2/tree/master/rknn-toolkit2/packages/x86_64
#you could use version 1.6.0 if you have problems with the latest toolkit version https://github.com/rockchip-linux/rknn-toolkit2/blob/master/rknn-toolkit2/packages
!python3 --version

Python 3.11.13


In [18]:
!wget https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/x86_64/requirements_cp311-2.3.2.txt
!wget https://github.com/airockchip/rknn-toolkit2/raw/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

--2025-07-24 20:49:53--  https://raw.githubusercontent.com/airockchip/rknn-toolkit2/refs/heads/master/rknn-toolkit2/packages/x86_64/requirements_cp311-2.3.2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360 [text/plain]
Saving to: ‘requirements_cp311-2.3.2.txt’

requirements_cp311- 100%[===================>]     360  --.-KB/s    in 0s      

2025-07-24 20:49:53 (7.15 MB/s) - ‘requirements_cp311-2.3.2.txt’ saved [360/360]

--2025-07-24 20:49:53--  https://github.com/airockchip/rknn-toolkit2/raw/refs/heads/master/rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sen

In [19]:
!pip3 install ./rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing ./rknn_toolkit2-2.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 654.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn as nn

# Define the PyTorch model
class MatrixMultiplication(nn.Module):
    def __init__(self):
        super(MatrixMultiplication, self).__init__()

    def forward(self, input1, input2):
        return torch.matmul(input1, input2)

# Instantiate the model
model = MatrixMultiplication()

# Create dummy inputs (batch size of 1, 9x9 matrices)
input1 = torch.randn(1, 9, 9)
input2 = torch.randn(1, 9, 9)

# Export to ONNX with opset_version 11 for RKNN compatibility
torch.onnx.export(model, (input1, input2), "matmul.onnx", opset_version=11)
print("PyTorch model exported to ONNX successfully.")

PyTorch model exported to ONNX successfully.


In [3]:
from rknn.api import RKNN

# Initialize RKNN
rknn = RKNN(verbose=True)

# Configure for RK3566
rknn.config(target_platform='rk3566')

# Load the ONNX model
ret = rknn.load_onnx(model='matmul.onnx')
if ret != 0:
    print('Load ONNX model failed')
    exit(ret)

# Build the model without quantization for simplicity
ret = rknn.build(do_quantization=False)
if ret != 0:
    print('Build model failed')
    exit(ret)

# Export the RKNN model
ret = rknn.export_rknn('matmul.rknn')
if ret != 0:
    print('Export RKNN model failed')
    exit(ret)

print("ONNX model converted to RKNN successfully.")

I rknn-toolkit2 version: 2.3.2
W load_onnx: The config.mean_values is None, zeros will be set for input 0!
W load_onnx: The config.std_values is None, ones will be set for input 0!
W load_onnx: The config.mean_values is None, zeros will be set for input 1!
W load_onnx: The config.std_values is None, ones will be set for input 1!
D base_optimize ...
D base_optimize done.
D 
D fold_constant ...
D fold_constant done.
D 
D correct_ops ...
D correct_ops done.
D 
D fuse_ops ...
D fuse_ops results:
D     convert_matmul_to_exmatmul: remove node = ['/MatMul'], add node = ['onnx::MatMul_0_tp', 'onnx::MatMul_0_tp_rs', 'onnx::MatMul_1_rs', '/MatMul#1', '2_mm_rs']
D     unsqueeze_to_4d_transpose: remove node = [], add node = ['onnx::MatMul_0_rs', 'onnx::MatMul_0_tp-rs']
D     bypass_two_reshape: remove node = ['onnx::MatMul_0_tp_rs', 'onnx::MatMul_0_tp-rs']
D     fold_constant ...
D     fold_constant done.
D fuse_ops done.
D 
D sparse_weight ...
D sparse_weight done.
D 
I rknn building ...
I rknn b

ONNX model converted to RKNN successfully.
